In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q datasets requests torch peft bitsandbytes transformers trl accelerate sentencepiece

In [ ]:
import os
import re
import math
from tqdm import tqdm
from google.colab import userdata
from huggingface_hub import login
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, set_seed
from peft import LoraConfig, PeftModel
from datetime import datetime

In [ ]:
import os
from os.path import join
import random
from tqdm.auto import tqdm
import requests

In [ ]:
!pip install -qU transformers==4.48.3 datasets==3.2.0 optimum==1.24.0
!pip install -qU openai==1.61.0 wandb

In [ ]:
BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B-Instruct"


In [ ]:
from google.colab import userdata
import wandb

wandb.login(key=userdata.get('wandb'))
hf_token = userdata.get('TOKEN_HUGINFACE')
!huggingface-cli login --token {hf_token}

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ahmed-ouka23456 (ahmed-ouka23456-ecole-nationale-de-l-intelligence-artifi) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `fine tuning` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `fine tuning`


In [ ]:
quant_config = BitsAndBytesConfig(load_in_8bit=True)

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=quant_config,
    device_map="cuda",
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
print(f"Memory footprint: {base_model.get_memory_footprint() / 1e9:,.1f} GB")

Memory footprint: 9.1 GB


In [ ]:
base_model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear8bitLt(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): Lla

In [ ]:
import json

In [ ]:
data_ar = []
with open('/content/drive/MyDrive/fine-tuning-PFA/data/aRsft.json', 'r') as f1:
    for line in f1:
        data_ar.append(json.loads(line))

data_xs = []
with open('/content/drive/MyDrive/fine-tuning-PFA/data/xsft.json', 'r') as f2:
    for line in f2:
        data_xs.append(json.loads(line))

In [ ]:
total_data = data_ar + data_xs
total_data[0]

{'id': 0,
 'question': 'من يمكنه الوصول إلى المباني التعليمية خارج الدروس؟ (البديل 12)',
 'task': "extraire des informations sur l'école",
 'output_scheme': '{"$defs": {"RelatedQuestion": {"properties": {"question1": {"description": "Question suggérée liée à la réponse et à la question posée", "title": "Question1", "type": "string"}, "question2": {"description": "Question suggérée liée à la réponse et à la question posée", "title": "Question2", "type": "string"}, "question3": {"description": "Question suggérée liée à la réponse et à la question posée", "title": "Question3", "type": "string"}}, "required": ["question1", "question2", "question3"], "title": "RelatedQuestion", "type": "object"}}, "properties": {"contentment": {"description": "Exprimez poliment que vous avez bien compris la question. Utilisez une phrase courte, rassurante, naturelle et adaptée au contexte de la question. Par exemple : \'Merci pour votre question\', \'Je comprends votre demande\', \'سؤال جيد، شكرًا لك\'. Rép

In [ ]:
import random

In [ ]:
llm_finetunning_data = []

system_message = "\n".join([
    "Vous êtes un analyseur de données professionnel en PNL. ",
    "Suivez les questions fournie par l'utilisateur et le schéma de sortie pour générer le fichier JSON de sortie. ",
    "Ne générez ni introduction ni conclusion.",
    ])

for item in total_data:
    rec = item

    llm_finetunning_data.append({
        "system": system_message,
        "instruction": "\n".join([
            "# question:",
            rec["question"],

            "# Task:",
            rec["task"],

            "# Output Scheme:",
            rec["output_scheme"],
            "",

            "# Output JSON:",
            "```json"

        ]),
        "input": "",
        "output": "\n".join([
            "```json",
            json.dumps(rec["response"], ensure_ascii=False, default=str),
            "```"
        ]),
        "history": []
    })

random.Random(101).shuffle(llm_finetunning_data)

In [ ]:
len(llm_finetunning_data)

4364

In [ ]:
data_dir = "/content/drive/MyDrive/fine-tuning-PFA/data"

In [ ]:
train_sample_sz = 4000

train_ds = llm_finetunning_data[:train_sample_sz]
eval_ds = llm_finetunning_data[train_sample_sz:]

os.makedirs(join(data_dir, "dataset_complet", "llamafactory-finetune-data"), exist_ok=True)

with open(join(data_dir, "dataset_complet", "llamafactory-finetune-data", "train.json"), "w") as dest:
    json.dump(train_ds, dest, ensure_ascii=False, default=str)

with open(join(data_dir, "dataset_complet", "llamafactory-finetune-data", "val.json"), "w", encoding="utf8") as dest:
    json.dump(eval_ds, dest, ensure_ascii=False, default=str)

In [ ]:
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
!cd LLaMA-Factory && pip install -e .

In [ ]:
#  "eniad-Assistant_finetune_train": {
#         "file_name": "/content/drive/MyDrive/fine-tuning-PFA/data/dataset_complet/llamafactory-finetune-data/train.json",
#         "columns": {
#             "prompt": "instruction",
#             "query": "input",
#             "response": "output",
#             "system": "system",
#             "history": "history"
#         }
#     },
#     "eniad-Assistant_finetune_val": {
#         "file_name": "/content/drive/MyDrive/fine-tuning-PFA/data/dataset_complet/llamafactory-finetune-data/val.json",
#         "columns": {
#             "prompt": "instruction",
#             "query": "input",
#             "response": "output",
#             "system": "system",
#             "history": "history"
#         }
#     }

In [ ]:
%%writefile /content/LLaMA-Factory/examples/train_lora/ENIAD-ASSISTANT_finetune.yaml

### model
model_name_or_path: meta-llama/Meta-Llama-3.1-8B
trust_remote_code: true


### method
stage: sft
do_train: true
finetuning_type: lora
lora_rank: 32
lora_target: [q_proj,k_proj,v_proj,o_proj]
### dataset
dataset: eniad-Assistant_finetune_train
eval_dataset: eniad-Assistant_finetune_val
template: llama3
cutoff_len: 3500
# max_samples: 50
overwrite_cache: true
preprocessing_num_workers: 16

### output
# resume_from_checkpoint: /content/drive/MyDrive/fine-tuning-PFA/model/checkpoint
output_dir: /content/drive/MyDrive/fine-tuning-PFA/model-llama/
logging_steps: 10
save_steps: 500
plot_loss: true
# overwrite_output_dir: true

### train
per_device_train_batch_size: 1
gradient_accumulation_steps: 4
learning_rate: 1.0e-4
num_train_epochs: 3.0
lr_scheduler_type: cosine
warmup_ratio: 0.1
bf16: true
ddp_timeout: 180000000
quantization_bit: 8

### eval
# val_size: 0.1
per_device_eval_batch_size: 1
eval_strategy: steps
eval_steps: 100
report_to: wandb
run_name: Eniad-assistant-finetune-llamafactory
push_to_hub: true
export_hub_model_id: "ahmed-ouka/ENIAD-ASSISTANT-llama3.1-8B"
hub_private_repo: true
hub_strategy: checkpoint


Overwriting /content/LLaMA-Factory/examples/train_lora/ENIAD-ASSISTANT_finetune.yaml


In [ ]:
!cd LLaMA-Factory/ && llamafactory-cli train /content/LLaMA-Factory/examples/train_lora/ENIAD-ASSISTANT_finetune.yaml

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
 65% 235/364 [01:29<00:48,  2.67it/s]
 65% 236/364 [01:29<00:48,  2.62it/s]
 65% 237/364 [01:30<00:48,  2.64it/s]
 65% 238/364 [01:30<00:48,  2.61it/s]
 66% 239/364 [01:30<00:48,  2.56it/s]
 66% 240/364 [01:31<00:48,  2.54it/s]
 66% 241/364 [01:31<00:47,  2.59it/s]
 66% 242/364 [01:31<00:46,  2.63it/s]
 67% 243/364 [01:32<00:46,  2.58it/s]
 67% 244/364 [01:32<00:46,  2.56it/s]
 67% 245/364 [01:33<00:45,  2.61it/s]
 68% 246/364 [01:33<00:44,  2.66it/s]
 68% 247/364 [01:33<00:44,  2.62it/s]
 68% 248/364 [01:34<00:43,  2.65it/s]
 68% 249/364 [01:34<00:43,  2.67it/s]
 69% 250/364 [01:35<00:43,  2.63it/s]
 69% 251/364 [01:35<00:43,  2.58it/s]
 69% 252/364 [01:35<00:41,  2.72it/s]
 70% 253/364 [01:36<00:40,  2.75it/s]
 70% 254/364 [01:36<00:39,  2.77it/s]
 70% 255/364 [01:36<00:40,  2.66it/s]
 70% 256/364 [01:37<00:39,  2.70it/s]
 71% 257/364 [01:37<00:40,  2.64it/s]
 71% 258/364 [01:37<00:38,  2.73it/s]
 71% 259/3

In [ ]:
finetuned_model_id = "/content/drive/MyDrive/fine-tuning-PFA/model-llama"
base_model.load_adapter(finetuned_model_id)

In [ ]:
base_model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): lora.Linear8bitLt(
            (base_layer): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Identity()
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=32, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=32, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear8bitLt(
            (base_layer): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(
              (def

In [ ]:
import json
import os
from os.path import join
import random
from tqdm.auto import tqdm
import requests

from pydantic import BaseModel, Field
from typing import List, Optional, Literal
from datetime import datetime


from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

In [ ]:
intent_category = Literal[
    "acadimique information", "Admissions et inscriptions", "Services aux étudiants",
    "Droits et responsabilités des étudiants", "Activités étudiantes", "Services administratifs",
    "Vie sur le campus", "Autre"
]
class RelatedQuestion(BaseModel):
    question1: str = Field(description="Question suggérée liée à la réponse et à la question posée" )
    question2: str = Field(description="Question suggérée liée à la réponse et à la question posée" )
    question3: str = Field(description="Question suggérée liée à la réponse et à la question posée" )

class ChatBotResponse(BaseModel):
    contentment: str = Field(
    ...,
    min_length=20,
    max_length=80,
    description=(
        "Exprimez poliment que vous avez bien compris la question. "
        "Utilisez une phrase courte, rassurante, naturelle et adaptée au contexte de la question. "
        "Par exemple : 'Merci pour votre question', 'Je comprends votre demande', 'سؤال جيد، شكرًا لك'. "
        "Répondez dans la même langue que celle de la question."
                 )
         )
    main_answer: str = Field(
    ...,
    min_length=20,
    max_length=100,
    description=(
        "Fournissez une réponse directe, claire et concise à la question posée. "
        "Évitez les longueurs inutiles. "
        "Répondez dans la même langue que celle utilisée dans la question."
                 )
           )
    details: Optional[str] = Field(
    None,
    min_length=20,
    max_length=500,
    description=(
        "Ajoutez des explications ou informations supplémentaires pour enrichir la réponse si nécessaire. "
        "Incluez des exemples, contextes ou précisions utiles. "
        "Répondez dans la même langue que celle de la question."
                 )
           )

    intent: intent_category = Field(
    ...,
    description=(
        "Identifiez l'intention principale de la question."
                )
           )

    related_questions: Optional[List[RelatedQuestion]] = Field(
    None,
    description=(
        "Une liste de questions similaires ou couramment posées en lien avec la question actuelle. "
        "Répondez dans la même langue que celle de la question d'origine."
                 )
             )

In [ ]:
prompte_Task_3_messages = [
     {
        "role": "system",
        "content": "\n".join([
            "Vous êtes un assistant dans une école qui s'appelle l'École Nationale de l'Intelligence Artificielle et Digitale de Berkane, capable de répondre aux questions des étudiants.",
            "Vous recevrez des informations sur l'école sous forme de JSON ainsi que des textes contenant des exemples de questions courantes et de leurs réponses.",
            "Vérifiez les données attentivement lorsque vous répondez.",
            "Essayez d'éviter tous les mots et textes qui n'ont pas de sens dans ces données.",
            "Faites très attention à la langue dans laquelle la question est posée.",
            "Vous devez répondre dans la même langue que celle de la question.",
            "Ne répondez pas tant que vous n’êtes pas sûr de la langue de la question.",
            "Si c'est en arabe, répondez en arabe. Si c'est en français, répondez en français. Si c'est en anglais, répondez en anglais.",
            "Ignorez les éléments inutiles dans la question tels que les numéros de version ou de commande, et concentrez-vous uniquement sur la question.",
            "Faites attention aux fautes d'orthographe pour ne pas altérer votre compréhension.",
            "Extraire les détails JSON du texte conformément aux questions posées et aux spécifications Pydantic.",
            "Extraire les détails comme indiqué dans le texte. Vous pouvez les reformater, mais gardez le sens.",
            "Ne pas générer d'introduction ni de conclusion.",
            "",


        ])
    },
    {
        "role": "user",
        "content": "\n".join([
            "## Question : {question}",



            "## Pydantic Details:",
            json.dumps(
                ChatBotResponse.model_json_schema(), ensure_ascii=False
            ),
            "",

            "## Repense Details:",
            "```json"
        ])
    }
]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
text = tokenizer.apply_chat_template(
    prompte_Task_3_messages,
    tokenize=False,
    add_generation_prompt=True
)

model_inputs = tokenizer([text], return_tensors="pt").to("cuda")

generated_ids = base_model.generate(
    model_inputs.input_ids,
    max_new_tokens=1024,
    do_sample=False, top_k=None, temperature=None, top_p=None,
)

generated_ids = [
    output_ids[len(input_ids):]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
print(response)

{"contentment": "سؤال جيد، شكرًا لك.", "main_answer": "المدرسة التي تتركز على الذكاء الاصطناعي والتقنيات الرقمية هي المدرسة الوطنية للذكاء الاصطناعي والتقنيات الرقمية في بركان.", "details": "تأسست المدرسة في [تاريخ التأسيس] وتهدف إلى تزويد الطلاب بالمهارات اللازمة في مجالات الذكاء الاصطناعي والتقنيات الرقمية.", "intent": "acadimique information", "related_questions": [{"question1": "ما هي مجالات الدراسة في المدرسة؟", "question2": "كيف يمكنني التسجيل في المدرسة؟", "question3": "ما هي المرافق المتاحة للطلاب في الحرم الجامعي؟"}]}


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig

In [ ]:
peft_config = PeftConfig.from_pretrained("/content/drive/MyDrive/fine-tuning-PFA/model-llama")
model = PeftModel.from_pretrained(base_model, "/content/drive/MyDrive/fine-tuning-PFA/model-llama")

In [ ]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
              (k_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
              (v_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
              (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
            )
            (mlp): LlamaMLP(
              (gate_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
              (up_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
              (down_proj): Linear8bitLt(in_features=14336, out_features=4096, bias=False)
              (act_fn): SiLU()
            )
            (input_layernorm): L

In [ ]:
merged_model = model.merge_and_unload()



/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:85: UserWarning: Merge lora module to 8-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
merged_model.save_pretrained("/content/drive/MyDrive/fine-tuning-PFA/model-llama-adapter", safe_serialization=True)

In [ ]:
merged_model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear8bitLt(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): Lla

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

In [ ]:
text = tokenizer.apply_chat_template(
    prompte_Task_3_messages,
    tokenize=False,
    add_generation_prompt=True
)

model_inputs = tokenizer([text], return_tensors="pt").to("cuda")

generated_ids = merged_model.generate(
    model_inputs.input_ids,
    max_new_tokens=1024,
    do_sample=False, top_k=None, temperature=None, top_p=None,
)

generated_ids = [
    output_ids[len(input_ids):]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [ ]:
print(response)

{"contentment": "سؤال جيد، شكرًا لك.", "main_answer": "المدرسة التي تعملك فيها هي المدرسة الوطنية للذكاء الاصطناعي والرقمي ببركان.", "details": "تأسست المدرسة في [تاريخ التأسيس] وتهدف إلى توفير التعليم والتدريب في مجالات الذكاء الاصطناعي والتقنيات الرقمية.", "intent": "acadimique information", "related_questions": [{"question1": "ما هي أهداف المدرسة؟", "question2": "كيف يمكنني التسجيل في المدرسة؟", "question3": "ما هي البرامج الدراسية المتاحة في المدرسة؟"}]}


In [ ]:
from huggingface_hub import HfApi
from huggingface_hub import create_repo, upload_folder
repo_id = "ahmed-ouka/Eniad-model-llama-Assistant"
api = HfApi()
create_repo(repo_id, private=False)
upload_folder(
    repo_id=repo_id,
    folder_path="/content/drive/MyDrive/fine-tuning-PFA/model-llama-adapter",
    path_in_repo=".",  # الجذر
    commit_message="model fine tuning for Eniad Assistant",
)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.09G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ahmed-ouka/Eniad-model-llama-Assistant/commit/4ca507d36c032abe503e1e1de145a5c074353d3b', commit_message='model fine tuning for Eniad Assistant', commit_description='', oid='4ca507d36c032abe503e1e1de145a5c074353d3b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ahmed-ouka/Eniad-model-llama-Assistant', endpoint='https://huggingface.co', repo_type='model', repo_id='ahmed-ouka/Eniad-model-llama-Assistant'), pr_revision=None, pr_num=None)

In [ ]:
merged_model.push_to_hub("ahmed-ouka/my-llama3.1-8B-with-lora-Eniad-Assistant")
tokenizer.push_to_hub("ahmed-ouka/my-llama3.1-8B-with-lora-Eniad-Assistant")

model-00002-of-00002.safetensors:   0%|          | 0.00/4.09G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ahmed-ouka/my-llama3.1-8B-with-lora-Eniad-Assistant/commit/1a5e2bc866014836269b965bd4d8129485bfc384', commit_message='Upload tokenizer', commit_description='', oid='1a5e2bc866014836269b965bd4d8129485bfc384', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ahmed-ouka/my-llama3.1-8B-with-lora-Eniad-Assistant', endpoint='https://huggingface.co', repo_type='model', repo_id='ahmed-ouka/my-llama3.1-8B-with-lora-Eniad-Assistant'), pr_revision=None, pr_num=None)